VERİ SETİ OLUŞTURMA

In [ ]:
import pandas as pd
import numpy as np

# Rastgele veri üretimi için sabit seed
np.random.seed(42)

# Veri seti oluşturma
n_samples = 500
data = {
    'hava_durumu': np.random.choice(['güneşli', 'yağmurlu', 'karlı'], n_samples, p=[0.6, 0.3, 0.1]),
    'uyku_suresi': np.random.normal(7, 1.5, n_samples).clip(4, 10),
    'motivasyon': np.random.randint(1, 11, n_samples),
    'mesafe': np.random.uniform(0.5, 20, n_samples).round(1),
    'gecmis_katilim': np.random.randint(50, 100, n_samples),
}

# Katılım durumu (hedef değişken) oluşturma
probability = (
   0.4 * (data['motivasyon'] + data['uyku_suresi'] > 10) +  # Yüksek motivasyon ve uyku süresi
    0.3 * (np.array(data['mesafe']) < 10) +                  # Mesafe yakın
    0.3 * (np.array(data['gecmis_katilim']) > 60)            # Geçmiş katılım oranı yüksek
)
# "geldi" ve "gelmedi" oranını dengelemek
class_ratio = 0.5  # "geldi" oranı
data['katilim'] = np.random.choice(
    ['geldi', 'gelmedi'],
    n_samples,
    p=[class_ratio, 1 - class_ratio]
)

# DataFrame oluşturma
df = pd.DataFrame(data)
df.to_csv('ogrenci_katilim.csv', index=False)
print("Veri seti başarıyla oluşturuldu ve 'ogrenci_katilim.csv' dosyasına kaydedildi!")
print(df['katilim'].value_counts())


Veri seti başarıyla oluşturuldu ve 'ogrenci_katilim.csv' dosyasına kaydedildi!
katilim
geldi      259
gelmedi    241
Name: count, dtype: int64


MODEL EĞİTİMİ VE KARŞILAŞTIRMA

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# AdaBoostClassifier'ı SAMME algoritmasıyla tanımlayın
model = AdaBoostClassifier(algorithm='SAMME', learning_rate=1.0)


# Veri yükleme
df = pd.read_csv('ogrenci_katilim.csv')

# Kategorik değişkenleri dönüştürme
encoder = OneHotEncoder()
hava_durumu_encoded = encoder.fit_transform(df[['hava_durumu']]).toarray()
hava_durumu_df = pd.DataFrame(hava_durumu_encoded, columns=encoder.get_feature_names_out(['hava_durumu']))

# Veri setini birleştirme
X = pd.concat([df[['uyku_suresi', 'motivasyon', 'mesafe', 'gecmis_katilim']], hava_durumu_df], axis=1)
y = df['katilim']

# Eğitim ve test ayrımı
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modellerin listesi
models = {
    "Logistic Regression": LogisticRegression(max_iter=200),
    "SVM": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Extra Trees": ExtraTreesClassifier(),
    "KNN": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "Neural Network": MLPClassifier(max_iter=300)
}

# Performans değerlendirme
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, pos_label='geldi')
    recall = recall_score(y_test, y_pred, pos_label='geldi')
    f1 = f1_score(y_test, y_pred, pos_label='geldi')
    results.append({"Model": name, "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1-Score": f1})

# Sonuçları DataFrame olarak görüntüleme
results_df = pd.DataFrame(results)
print(results_df.sort_values(by="F1-Score", ascending=False))


                 Model  Accuracy  Precision    Recall  F1-Score
1                  SVM      0.55   0.550000  1.000000  0.709677
0  Logistic Regression      0.53   0.571429  0.581818  0.576577
9       Neural Network      0.48   0.523077  0.618182  0.566667
6          Extra Trees      0.52   0.566038  0.545455  0.555556
2        Decision Tree      0.51   0.557692  0.527273  0.542056
3        Random Forest      0.49   0.535714  0.545455  0.540541
7                  KNN      0.47   0.518519  0.509091  0.513761
8          Naive Bayes      0.42   0.476190  0.545455  0.508475
4    Gradient Boosting      0.46   0.509434  0.490909  0.500000
5             AdaBoost      0.43   0.482143  0.490909  0.486486


MODELİN KAYDEDİLMESİ

In [32]:
import joblib
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Veri yükleme
df = pd.read_csv('ogrenci_katilim.csv')

# Kategorik değişkenleri dönüştürme
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
hava_durumu_encoded = encoder.fit_transform(df[['hava_durumu']]).toarray()
hava_durumu_df = pd.DataFrame(hava_durumu_encoded, columns=encoder.get_feature_names_out(['hava_durumu']))

# Veri setini birleştirme
X = pd.concat([df[['uyku_suresi', 'motivasyon', 'mesafe', 'gecmis_katilim']], hava_durumu_df], axis=1)
y = df['katilim']

# Eğitim ve test ayrımı
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# SVM modelini oluşturma
svm_model = SVC(kernel='linear')  # Linear kernel kullanarak SVM modelini oluşturuyoruz
svm_model.fit(X_train, y_train)

# Modelin performansını değerlendirme
y_pred = svm_model.predict(X_test)
print("SVM Model Performansı:")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred, pos_label='geldi')}")
print(f"Recall: {recall_score(y_test, y_pred, pos_label='geldi')}")
print(f"F1-Score: {f1_score(y_test, y_pred, pos_label='geldi')}")

# Modeli kaydetme
joblib.dump(svm_model, 'svm_model.joblib')


SVM Model Performansı:
Accuracy: 0.52
Precision: 0.5660377358490566
Recall: 0.5454545454545454
F1-Score: 0.5555555555555556


['svm_model.joblib']